In [1]:
import os
import sys
libdir = os.path.dirname('/usr/people/muntjewe/git/laura_scripts/')
sys.path.insert(0, libdir) 

from libpyvarex import libtimeseries

In [2]:
import numpy as np
import xarray as xr

### Open the data from LENTIS PD and from 2K 

In [3]:
ds_tas_PD=xr.open_mfdataset('/usr/people/muntjewe/nobackup/nobackup_1/LENTIS/PD/Amon/tas/tas*.nc',combine='nested',concat_dim='ens')
tas_PD=ds_tas_PD['tas']

In [4]:
ds_tas_2K=xr.open_mfdataset('/usr/people/muntjewe/nobackup/nobackup_1/LENTIS/2K/Amon/tas/tas*.nc',combine='nested',concat_dim='ens')
tas_2K=ds_tas_2K['tas']

# align the time dimension, necessary for subtracting 2 xarray datasets
to check if it worked, run the following (this should not give an error): 
`xr.align(tas_PD, tas_2K, join='exact')`

In [ ]:
tas_2K['time'] = tas_PD['time']

### compute and print seasonal mean and std dev of the differences

In [11]:

diff_season = tas_2K - tas_PD 
diff_mean = diff_season.mean(dim='ens').groupby('time.season').mean(dim='time')
diff_std = diff_season.std(dim='ens').groupby('time.season').mean(dim='time')

In [14]:
print('Now we are computing some statistics of the tas differences between PD an 2K (takes a long time!!')



print('Global')
## Explicit conversion by wrapping a DataArray with np.asarray also works. I need to do this to access values. See https://docs.xarray.dev/en/v0.9.3/dask.html
glbias = (np.asarray(libtimeseries.boxmean(diff_mean))) 
glstd = (np.asarray(libtimeseries.boxmean(diff_std))) 
print(str(libtimeseries.boxmean(diff_mean).season[0].values)+' = '+str(np.round(glbias[0],2))+' (ens stddev: '+str(np.round(glstd[0],2))+')')
print(str(libtimeseries.boxmean(diff_mean).season[1].values)+' = '+str(np.round(glbias[1],2))+' (ens stddev: '+str(np.round(glstd[1],2))+')')
print(str(libtimeseries.boxmean(diff_mean).season[2].values)+' = '+str(np.round(glbias[2],2))+' (ens stddev: '+str(np.round(glstd[2],2))+')')
print(str(libtimeseries.boxmean(diff_mean).season[3].values)+' = '+str(np.round(glbias[3],2))+' (ens stddev: '+str(np.round(glstd[3],2))+')')
print('ANN = '+str(np.round(glbias.mean(),2))+' (ens stddev: '+str(np.round(glstd.mean(),2))+')')
print('')
nhbias = (np.asarray(libtimeseries.NHboxmean(diff_mean))) 
nhstd = (np.asarray(libtimeseries.NHboxmean(diff_std))) 
print('Northern Hemisphere')
print(str(libtimeseries.boxmean(diff_mean).season[0].values)+' = '+str(np.round(nhbias[0],2))+' (ens stddev: '+str(np.round(nhstd[0],2))+')')
print(str(libtimeseries.boxmean(diff_mean).season[1].values)+' = '+str(np.round(nhbias[1],2))+' (ens stddev: '+str(np.round(nhstd[1],2))+')')
print(str(libtimeseries.boxmean(diff_mean).season[2].values)+' = '+str(np.round(nhbias[2],2))+' (ens stddev: '+str(np.round(nhstd[2],2))+')')
print(str(libtimeseries.boxmean(diff_mean).season[3].values)+' = '+str(np.round(nhbias[3],2))+' (ens stddev: '+str(np.round(nhstd[3],2))+')')
print('== ANN: '+str(np.round(nhbias.mean(),2))+' (ens stddev: '+str(np.round(nhbias.mean(),2))+')')
print('')
eubias = (np.asarray(libtimeseries.EUboxmean(diff_mean))) 
eustd = (np.asarray(libtimeseries.EUboxmean(diff_std))) 
print('Europe')
print(str(libtimeseries.boxmean(diff_mean).season[0].values)+' = '+str(np.round(eubias[0],2))+' (ens stddev: '+str(np.round(eustd[0],2))+')')
print(str(libtimeseries.boxmean(diff_mean).season[1].values)+' = '+str(np.round(eubias[1],2))+' (ens stddev: '+str(np.round(eustd[1],2))+')')
print(str(libtimeseries.boxmean(diff_mean).season[2].values)+' = '+str(np.round(eubias[2],2))+' (ens stddev: '+str(np.round(eustd[2],2))+')')
print(str(libtimeseries.boxmean(diff_mean).season[3].values)+' = '+str(np.round(eubias[3],2))+' (ens stddev: '+str(np.round(eustd[3],2))+')')
print('== ANN: '+str(np.round(eubias.mean(),2))+' (ens stddev: '+str(np.round(eubias.mean(),2))+')')

Now we are computing some statistics of the tas differences between PD an 2K
Global biases
DJF = 2.02 (ens stddev: 1.51)
JJA = 1.9 (ens stddev: 1.23)
MAM = 1.83 (ens stddev: 1.39)
SON = 2.06 (ens stddev: 1.26)
ANN = 1.95 (ens stddev: 1.35)

Northern Hemisphere biases
DJF = 2.67 (ens stddev: 1.99)
JJA = 2.42 (ens stddev: 1.37)
MAM = 2.29 (ens stddev: 1.72)
SON = 2.72 (ens stddev: 1.49)
== ANN: 2.52 (ens stddev: 2.52)

Europe biases
DJF = 2.4 (ens stddev: 2.58)
JJA = 2.81 (ens stddev: 1.86)
MAM = 2.29 (ens stddev: 2.16)
SON = 2.69 (ens stddev: 1.9)
== ANN: 2.55 (ens stddev: 2.55)
